In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.utils import plot_model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
x_train = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/x_train.npy")/255.
x_test = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/x_test.npy")/255.
y_train_coarse = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/y_train_fine.npy")

In [ ]:
import pickle
with open("/kaggle/input/itba-rn-2245-cifar-100-2023q2/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/itba-rn-2245-cifar-100-2023q2/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [ ]:
from matplotlib import pyplot as plt
%load_ext tensorboard

In [ ]:
plt.figure(figsize=(1,1))
plt.imshow(x_train[0])

In [ ]:
labels_coarse[11]

In [ ]:
# Me fijo para una categoría de grano grueso, cuáles son las categorías de grano fino
coarse = 11
print(f"Grano grueso: {labels_coarse[coarse]}")
for fine in np.unique(y_train_fine[y_train_coarse == coarse]):
    print(f"Grano fino: {labels_fine[fine]}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard

Flatten es la capa de entrada

El relu es la hidden layer -> 

Softmax -> Capa de salida transforma los números a probabilidades

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(32,32,3)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(100, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", metrics = ["accuracy"], optimizer = Adam(learning_rate=0.001))

In [ ]:
model.summary()

In [ ]:
plot_model(model,show_shapes=True,show_layer_names=True)

In [ ]:
rlrop = ReduceLROnPlateau(
    monitor = "val_accuracy",
    factor = 0.5,
    patience = 4,
    verbose = 1,
    min_lr = 1e-5
)

In [ ]:
mc = ModelCheckpoint(
    "best_weights.hfd5",
    monitor = "val_accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
)

In [ ]:
es = EarlyStopping(
    monitor = "val_accuracy",
    patience = 8, 
    verbose = 1,
    restore_best_weights = True,
)

In [ ]:
tb = TensorBoard(
    log_dir="logs",
)

In [ ]:
history = model.fit(
    x = x_train, 
    y = y_train_fine, 
    batch_size=128, 
    epochs=200, 
    validation_split = 0.4, 
    callbacks= [
        rlrop,
        es,
        mc,
        tb,
    ]
)

In [ ]:
plt.plot(history.history["loss"], label='loss')
plt.plot(history.history["val_loss"], label='val_loss')
plt.legend()

In [ ]:
plt.plot(history.history["accuracy"], label='accuracy')
plt.plot(history.history["val_accuracy"], label='val_accuracy')
plt.legend()

In [ ]:
%tensorboard --logdir logs

In [ ]:
predictions = model.predict(x_test).argmax(axis=1)

In [ ]:
df = pd.DataFrame(predictions, columns=["Label"])

In [ ]:
df.index.name = "Id"

In [ ]:
df.head()

In [ ]:
df.to_csv("submission.csv")